In [3]:
import sys, os

module_path = os.path.abspath(os.getcwd() + '/../../..')
print(module_path)

os.environ["OPENAI_API_KEY"] = ""

sys.path.append(module_path)

/home/jupyter/green.ai


In [4]:
import os
import asyncio
import subprocess

from deepeval.dataset import Golden
from deepeval.test_case import LLMTestCase
from openai import AsyncOpenAI, OpenAI

from benchmarq.utility import VLLMServerSingleton
from benchmarq.utility import Evaluator
from benchmarq.experiment import Experiment

from dotenv import load_dotenv

load_dotenv()

csv_files = ["100",
             "500",
             "1000",
             "2000",
             "3000",
             "4000",
             "5000",
              ]

debug = False
model = "EleutherAI/pythia-1.4b"

#server = VLLMServerSingleton()

#server = VLLMServerSingleton()

#if not debug:
#    server.start_server(model=model)

openai_api_key = os.environ["OPENAI_API_KEY"]
openai_api_base = "http://localhost:8000/v1"

async_client = AsyncOpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

class Test(Evaluator):
    def __init__(self):
        super().__init__()

    async def async_evaluate_consumption(self, input: Golden):
        return await async_client.completions.create(
            model=model,
            prompt=f"{input.context} {input.input}",
            max_tokens=50
        )

    async def evaluate_test_case(self, input: Golden) -> LLMTestCase:
        output = await async_client.completions.create(
            model=model,
            prompt=f"{input.context} {input.input}",
            max_tokens=50
        ).choices[0].message.content

        return LLMTestCase(
            input=input.input,
            expected_output=input.expected_output,
            actual_output=output,
            context=input.context,
            retrieval_context=input.retrieval_context)

for dataset in csv_files:
    experiment = Experiment(
        subquestion_id="input_size",
        id=dataset,
        dataset_name=dataset,
        subquestion_path="experiments/context_window/input_size/tests.json",
        name=dataset,
        description=f"A test to see if energy readings change based on input size. Current dataset size: {dataset}",
        settings=Test(),
        skip_metrics=True,
        debug_mode=debug,
        )
    a = await experiment.run()


#if not debug:
#    server.stop_server()

[codecarbon INFO @ 13:39:52] [setup] RAM Tracking...
[codecarbon INFO @ 13:39:52] [setup] GPU Tracking...
[codecarbon INFO @ 13:39:52] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:39:52] [setup] CPU Tracking...
[codecarbon WARNING @ 13:39:52] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 13:39:53] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 13:39:53] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 13:39:53] >>> Tracker's metadata:
[codecarbon INFO @ 13:39:53]   Platform system: Linux-5.10.0-34-cloud-amd64-x86_64-with-glibc2.31
[codecarbon INFO @ 13:39:53]   Python version: 3.11.0
[codecarbon INFO @ 13:39:53]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 13:39:53]   Available RAM : 14.649 GB
[codecarbon INFO @ 13:39:53]   CPU count: 4
[codecarbon INFO @ 13:39:53]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 13:3

warmup for: 100


100%|██████████| 100/100 [00:02<00:00, 37.14it/s]


testing: 100


100%|██████████| 100/100 [00:04<00:00, 23.52it/s]
[codecarbon INFO @ 13:40:00] Energy consumed for RAM : 0.000006 kWh. RAM Power : 5.493237018585205 W
[codecarbon INFO @ 13:40:00] Energy consumed for all GPUs : 0.000058 kWh. Total GPU Power : 49.057 W
[codecarbon INFO @ 13:40:00] Energy consumed for all CPUs : 0.000051 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:40:00] 0.000115 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:40:01] [setup] RAM Tracking...
[codecarbon INFO @ 13:40:01] [setup] GPU Tracking...
[codecarbon INFO @ 13:40:01] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:40:01] [setup] CPU Tracking...
[codecarbon WARNING @ 13:40:01] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 13:40:02] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 13:40:02] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 13:40:

...

  1%|          | 6/600 [00:14<19:43,  1.99s/it][codecarbon INFO @ 13:43:55] Energy consumed for RAM : 0.000023 kWh. RAM Power : 5.493237018585205 W
[codecarbon INFO @ 13:43:55] Energy consumed for all GPUs : 0.000322 kWh. Total GPU Power : 77.2 W
[codecarbon INFO @ 13:43:55] Energy consumed for all CPUs : 0.000178 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:43:55] 0.000522 kWh of electricity used since the beginning.
 27%|██▋       | 163/600 [00:26<01:08,  6.35it/s][codecarbon INFO @ 13:44:10] Energy consumed for RAM : 0.000046 kWh. RAM Power : 5.493237018585205 W
[codecarbon INFO @ 13:44:10] Energy consumed for all GPUs : 0.000622 kWh. Total GPU Power : 72.188 W
[codecarbon INFO @ 13:44:10] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:44:10] 0.001022 kWh of electricity used since the beginning.
 76%|███████▋  | 458/600 [00:38<00:01, 124.36it/s][codecarbon INFO @ 13:44:25] Energy consumed for RAM : 0.000069 kWh. RAM Power : 5.49323

warmup for: 5000


100%|██████████| 700/700 [01:12<00:00,  9.71it/s]


testing: 5000


  7%|▋         | 50/700 [00:11<01:27,  7.43it/s][codecarbon INFO @ 13:46:02] Energy consumed for RAM : 0.000023 kWh. RAM Power : 5.493237018585205 W
[codecarbon INFO @ 13:46:02] Energy consumed for all GPUs : 0.000335 kWh. Total GPU Power : 80.483 W
[codecarbon INFO @ 13:46:02] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:46:02] 0.000536 kWh of electricity used since the beginning.
 58%|█████▊    | 404/700 [00:29<00:23, 12.59it/s][codecarbon INFO @ 13:46:17] Energy consumed for RAM : 0.000046 kWh. RAM Power : 5.493237018585205 W
[codecarbon INFO @ 13:46:17] Energy consumed for all GPUs : 0.000654 kWh. Total GPU Power : 76.636 W
[codecarbon INFO @ 13:46:17] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:46:17] 0.001054 kWh of electricity used since the beginning.
 72%|███████▏  | 506/700 [00:44<00:30,  6.34it/s][codecarbon INFO @ 13:46:32] Energy consumed for RAM : 0.000069 kWh. RAM Power : 5.493

In [ ]:
from benchmarq.utility import VLLMServerSingleton


server = VLLMServerSingleton()


server.stop_server()